In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)
import glob
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from tensorflow.keras.layers import Reshape, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras import backend 


In [8]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]
sentiments = np.array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

In [9]:
#Embedding Layer (wordVocab,wordDim,inputlen) -- transforms words as rows, dimesions as cols matrx
from nltk.tokenize import word_tokenize

all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    #tokenize_word is a list of all words in a sentence
    for word in tokenize_word:
        all_words.append(word)
#all_words
#we only need unique words 
unique_words = set(all_words)
print(len(unique_words))
vocab_length = len(unique_words)

45


In [10]:
from tensorflow.keras.preprocessing.text import one_hot
embedded_sentences = [one_hot(sent, vocab_length) for sent in corpus]
print(embedded_sentences)
#but all these are different lengths so pad them equal to len if longest sentence

[[20, 17, 7, 43, 6], [9, 8, 14, 2, 32, 14, 6], [21, 39, 12, 6, 17, 20], [20, 21], [13, 18, 44, 10, 32, 14, 6], [24, 3, 2, 1], [24, 21, 34, 3, 33, 6], [6, 17, 3, 34, 6], [17, 33], [5, 40, 34], [34, 31], [6, 14, 38, 12], [32, 39, 20, 14, 9, 6], [9, 6, 14, 17], [32, 12, 20, 18], [9, 33, 17, 34]]


In [16]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

In [23]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[20 17  7 43  6  0  0]
 [ 9  8 14  2 32 14  6]
 [21 39 12  6 17 20  0]
 [20 21  0  0  0  0  0]
 [13 18 44 10 32 14  6]
 [24  3  2  1  0  0  0]
 [24 21 34  3 33  6  0]
 [ 6 17  3 34  6  0  0]
 [17 33  0  0  0  0  0]
 [ 5 40 34  0  0  0  0]
 [34 31  0  0  0  0  0]
 [ 6 14 38 12  0  0  0]
 [32 39 20 14  9  6  0]
 [ 9  6 14 17  0  0  0]
 [32 12 20 18  0  0  0]
 [ 9 33 17 34  0  0  0]]


In [24]:
#takes the input sentences and transforms to a higher dim, 20 is dim in this case
e = Embedding(vocab_length, 20, input_length=length_long_sentence)
e.output_dim #16 * 7* 20

20

In [25]:
from tensorflow.keras import Sequential
model = Sequential()
model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [26]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 7, 20)             900       
_________________________________________________________________
flatten_2 (Flatten)          (None, 140)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 141       
Total params: 1,041
Trainable params: 1,041
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 1ms/step - loss: 0.7020 - acc: 0.3125
Epoch 2/100
1/1 [==============================] - 0s 0s/step - loss: 0.6985 - acc: 0.4375
Epoch 3/100
1/1 [==============================] - 0s 0s/step - loss: 0.6951 - acc: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 997us/step - loss: 0.6917 - acc: 0.5625
Epoch 5/100
1/1 [==============================] - 0s 998us/step - loss: 0.6883 - acc: 0.5625
Epoch 6/100
1/1 [==============================] - 0s 0s/step - loss: 0.6850 - acc: 0.6875
Epoch 7/100
1/1 [==============================] - 0s 998us/step - loss: 0.6816 - acc: 0.6875
Epoch 8/100
1/1 [==============================] - 0s 998us/step - loss: 0.6783 - acc: 0.7500
Epoch 9/100
1/1 [==============================] - 0s 0s/step - loss: 0.6749 - acc: 0.7500
Epoch 10/100
1/1 [==============================] - 0s 998us/step - loss: 0.6715 - acc: 0.7500
Epoch 11/100
1/1 [==============================] - 0s 0s/step - loss: 0.

1/1 [==============================] - 0s 0s/step - loss: 0.3412 - acc: 0.9375
Epoch 89/100
1/1 [==============================] - 0s 0s/step - loss: 0.3370 - acc: 0.9375
Epoch 90/100
1/1 [==============================] - 0s 1000us/step - loss: 0.3328 - acc: 0.9375
Epoch 91/100
1/1 [==============================] - 0s 998us/step - loss: 0.3287 - acc: 0.9375
Epoch 92/100
1/1 [==============================] - 0s 995us/step - loss: 0.3246 - acc: 0.9375
Epoch 93/100
1/1 [==============================] - 0s 996us/step - loss: 0.3205 - acc: 0.9375
Epoch 94/100
1/1 [==============================] - 0s 0s/step - loss: 0.3165 - acc: 0.9375
Epoch 95/100
1/1 [==============================] - 0s 0s/step - loss: 0.3125 - acc: 0.9375
Epoch 96/100
1/1 [==============================] - 0s 998us/step - loss: 0.3085 - acc: 0.9375
Epoch 97/100
1/1 [==============================] - 0s 998us/step - loss: 0.3046 - acc: 0.9375
Epoch 98/100
1/1 [==============================] - 0s 997us/step - loss: 

In [28]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 93.750000
